In [1]:
from operator import index
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [2]:
import pandas as pd
import requests
import json
import pprint


/Users/apabook/Desktop/ds-deployment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Testing HL Api for PULL transfer

---

## Consts

In [4]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
api_test = "http://api-test:3000"


catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [5]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '11091a69-c681-4940-ae6d-6831dee4da2c',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '11091a69-c681-4940-ae6d-6831dee4da2c',
 'dct:issued': '2024-12-02T14:24:53.922615',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [6]:
payload = {
    "dcat:endpointURL": api_test
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '4be51801-e948-475e-8573-132910f2b3c8',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '',
 'dcat:endpointURL': 'http://api-test:3000',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '4be51801-e948-475e-8573-132910f2b3c8',
 'dct:issued': '2024-12-02T14:24:54.893423',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [7]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': 'a2cfe271-35c2-4920-87cf-52ef35529149',
 'data_service_id': '4be51801-e948-475e-8573-132910f2b3c8',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [8]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/6751f580-61f9-4faf-8939-a516965a5d44',
 'callbackId': '6751f580-61f9-4faf-8939-a516965a5d44',
 'consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1'}


## Request transfer

In [9]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
  "agreementId": agreement_pid,
  "format": "http+pull",
  "dataAddress": None,
  "callbackAddress": callbackAddress,
  "callbackId": callbackId,
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
                     'dspace:providerPid': 'urn:uuid:b7e571b6-40fa-42b5-96fd-905192d824fe',
                     'dspace:state': 'dspace:REQUESTED'}}


## Get data address

In [10]:
import requests

consumerPidUuid = consumerPid.replace("urn:uuid:", "")
url = data_space_consumer + "/api/v1/data-address/" + consumerPidUuid

payload = ""
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
dataPlaneAddress = response.json()["dataPlaneAddress"]

pprint.pprint(response.json())

{'dataPlaneAddress': 'http://ds-consumer:1235/6751f580-61f9-4faf-8939-a516965a5d44/data/pull/c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1'}


## Use data address to get data from data space

In [11]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


## Suspend transfer

In [12]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
                     'dspace:providerPid': 'urn:uuid:b7e571b6-40fa-42b5-96fd-905192d824fe',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended

**Disclaimer**: Data shouldn't appear. But for this deployment does appear because an internal refactor we're facing.
In a couple of days this functionality will work like is expected.

In [13]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

HTTPError: HTTP Error 400: Bad Request

## Restart transfer

In [14]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
                     'dspace:providerPid': 'urn:uuid:b7e571b6-40fa-42b5-96fd-905192d824fe',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

In [15]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


## Complete transfer

In [16]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:c83d7539-89ad-4e4d-8cdc-2608f1e2d7b1',
                     'dspace:providerPid': 'urn:uuid:b7e571b6-40fa-42b5-96fd-905192d824fe',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

In [17]:
url = dataPlaneAddress
url = url.replace("ds-consumer", "127.0.0.1")
df = pd.read_json(url + "/comments")
df

HTTPError: HTTP Error 400: Bad Request

## Protocol state cannot change

In [195]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:088d0bd9-2252-49e9-8809-0790f90c1a95',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
